In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

> notebook Sample code learning

In [ ]:
# import define
# os:운영체제에서 제공되는 여러 기능을 파이썬에서 수행할 수있게 함
# json: Json 데이터를 처리하기 위해서 사용되는 파이썬 내장 모듈
# numpy(np): 고성능의 수치계산. 대규모 다차원 배열과 행렬 연산에 필요한 다양한 함수 제공
# pandas(pd): 데이터분석 라이브러리로 행과열로 이루어진 데이터 객체를 만들어 다룸.
# seaborn(sn): 데이터프레임으로 다양한 통계 지표를 낼수 있는 시각화 차트를 제공
# matplotlib.pyolot(plt): 데이터를 차트나 플롯(Plot)으로 그려주는 데이터 시각화
#                       라인플롯, 바 차트, 파이차트, 히스토그램, BoxPlot 등
# cv2: openCV 컴퓨터 비전 라이브러리로 객체 얼굴 행동 모션 추적 등의 응용에서 사용
# albumentations:  이미지 형태 변환
# 
import os
import json

import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import cv2
import albumentations as A
from sklearn import metrics as sk_metrics

In [ ]:
# Kaggle base diretory def

BASE_DIR = "../input/cassava-leaf-disease-classification/"

In [ ]:
#os.path.join: 경로를 병합하여 새 경로를 생성
with open(os.path.join(BASE_DIR, "label_num_to_disease_map.json")) as file:
    
    #file 데이터를 json.load()를 사용하여 객체로 읽어오기 -> map_classes
    map_classes = json.loads(file.read())
    
    # for var in 문자열(튜플, 문자열)
    map_classes = {int(k) : v for k, v in map_classes.items()}

    #json.dumpsL 객체를 json 데이터로 쓰기, 직렬화 ,인코딩
print(json.dumps(map_classes, indent=4))

In [ ]:
#os.listdir 해당 디렉토리에 있는 파일들의 리스트를 구함
input_files = os.listdir(os.path.join(BASE_DIR, "train_images"))

#len() 리스트에 들어 있는 원소의 갯수
print(f"Number of train images: {len(input_files)}")

In [ ]:
img_shapes = {}
for image_name in os.listdir(os.path.join(BASE_DIR, "train_images"))[:300]:
    
    #cv2.imread: 함수를 이용하여 이미지 파일을 읽음
    image = cv2.imread(os.path.join(BASE_DIR, "train_images", image_name))
    #shape image 변수의 이미지 shape를 확인(height, width, channl)
    img_shapes[image.shape] = img_shapes.get(image.shape, 0) + 1

print(img_shapes)

In [ ]:
#pandas library의 read_csv() 함수를 이용하여 train.csv 함수를 읽음
df_train = pd.read_csv(os.path.join(BASE_DIR, "train.csv"))

#map은 리스트의 요소를 지정된 함수로 처리해주는 함수
# list(map(함수, 리스트))
# https://dojang.io/pluginfile.php/13699/mod_page/content/3/022019.png
df_train["class_name"] = df_train["label"].map(map_classes)

df_train

In [ ]:
#figure() 함수는 matplotlib에서 figure를 만들고 편집할 수 있게 만들어주는 함수
# ex1) fig=figure(num=figure number, figsize = (x,y))
# ex2) fig=figure(figure number, (x,y))
# ex3) fig=girure()
 plt.figure(figsize=(8, 4))

#countplot 카테고리별 데이터 양 확인 
sn.countplot(y="class_name", data=df_train);

In [ ]:
plt.figure(figsize=(4, 10))

#countplot 카테고리별 데이터 양 확인 
sn.countplot(x="class_name", data=df_train);

In [ ]:
# define visualize_batch 
def visualize_batch(image_ids, labels):
    plt.figure(figsize=(16, 12))
    
    #enumerate :반복문 사용시 몇번째 반복문인지 확인이 필요할 수 있음
    #인덱스 번호와 컬렉션의 원소를 tuple 형태로 반환
    
    for ind, (image_id, label) in enumerate(zip(image_ids, labels)):
        
        #subplot 한 화면에 여러 그래프를 나눠서 그려주는 기능
        plt.subplot(3, 3, ind + 1)
        image = cv2.imread(os.path.join(BASE_DIR, "train_images", image_id))
        
        #open cv에는 color-space를 변환방법이 약 150가지 있음
        #cv2.cvtColor( image_src , color_code )
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        #imshow() 이미지를 모니터에 출력
        plt.imshow(image)
        plt.title(f"Class: {label}", fontsize=12)
        # x,y축의 범위를 설정할수 있게 하는것과 동시에 여러 옵션을 설정할 수 있는 함수
        plt.axis("off")
    
    plt.show()

In [ ]:
#smaple(컬랙션, 샘플수) 지정된 컬렉션으로 부터 샘플수 만큼 추출
tmp_df = df_train.sample(9)

image_ids = tmp_df["image_id"].values
labels = tmp_df["class_name"].values

visualize_batch(image_ids, labels)

In [ ]:
#smaple(컬랙션, 샘플수) 지정된 컬렉션으로 부터 샘플수 만큼 추출
tmp_df = df_train.sample(6)

image_ids = tmp_df["image_id"].values
labels = tmp_df["class_name"].values

visualize_batch(image_ids, labels)

* 1. 0 - CBB - Cassava Bacterial Blight(카사바 박테리아 병균)


In [ ]:
tmp_df = df_train[df_train["label"] == 0]
print(f"Total train images for class 0: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image_id"].values
labels = tmp_df["label"].values

visualize_batch(image_ids, labels)

* 1 - CBSD - Cassava Brown Streak Disease
* 

In [ ]:
tmp_df = df_train[df_train["label"] == 1]
print(f"Total train images for class 1: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image_id"].values
labels = tmp_df["label"].values

visualize_batch(image_ids, labels)

* 2 - CGM - Cassava Green Mottle


In [ ]:
tmp_df = df_train[df_train["label"] == 2]
print(f"Total train images for class 2: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image_id"].values
labels = tmp_df["label"].values

visualize_batch(image_ids, labels)

* 3 - CMD - Cassava Mosaic Disease
* 

In [ ]:
tmp_df = df_train[df_train["label"] == 3]
print(f"Total train images for class 3: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image_id"].values
labels = tmp_df["label"].values

visualize_batch(image_ids, labels)

* 4 - Healthy
* 

In [ ]:
tmp_df = df_train[df_train["label"] == 4]
print(f"Total train images for class 4: {tmp_df.shape[0]}")

tmp_df = tmp_df.sample(9)
image_ids = tmp_df["image_id"].values
labels = tmp_df["label"].values

visualize_batch(image_ids, labels)
